In [9]:
# Main Block For EXP 2 3 5
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset (replace with your actual file path)
data = pd.read_csv('SolarPrediction.csv')

# Drop unnecessary columns like Time, UNIXTime (if not required for prediction)
data = data.drop(columns=['Time', 'TimeSunRise', 'TimeSunSet', 'UNIXTime', 'Data'])

# Check for missing values and drop or fill them
data = data.dropna()  # Drop rows with missing values

# Split data into features (X) and target (y)
X = data.drop(columns=['Radiation'])  # Assuming Radiation is the target
y = data['Radiation']

# Convert 'Radiation' into binary (0 or 1), threshold at 0.5
threshold = 300
y = np.where(y >= threshold, 1, 0)

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled[:5])
print(y[:5])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


[[-0.50043927  0.67897416 -0.61625306  0.40761954 -0.17873758]
 [-0.50043927  0.67897416 -0.65472966  0.40028483 -0.82335911]
 [-0.50043927  0.67897416 -0.69320626  0.1834901  -0.82335911]
 [-0.50043927  0.67897416 -0.57777646 -0.06949721 -0.82335911]
 [-0.50043927  0.67897416 -0.50082325 -0.46340711 -0.17873758]]
[0 0 0 0 0]


In [17]:
# EXP 5 single
class SingleLayerPerceptron:
    def __init__(self, learning_rate=0.01, n_iter=1000):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Initialize weights and bias
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Training the model
        for iteration in range(self.n_iter):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_function(linear_output)

                # Update weights and bias
                update = self.learning_rate * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

            # Debug: Print loss every 100 iterations
            if iteration % 100 == 0:
                loss = np.mean((y - self.predict(X)) ** 2)  # Mean Squared Error
                print(f"Iteration {iteration}, Loss: {loss:.4f}")

    def activation_function(self, x):
        return np.where(x >= 0, 1, 0)

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return self.activation_function(linear_output)

# Instantiate and train SLP
slp = SingleLayerPerceptron(learning_rate=0.01, n_iter=1000)
slp.fit(X_train, y_train)

# Predictions
slp_predictions = slp.predict(X_test)
slp_accuracy = np.mean(slp_predictions == y_test)
print(f'Single Layer Perceptron Accuracy: {slp_accuracy:.2f}')


Iteration 0, Loss: 0.1677
Iteration 100, Loss: 0.1637
Iteration 200, Loss: 0.1524
Iteration 300, Loss: 0.1859
Iteration 400, Loss: 0.1637
Iteration 500, Loss: 0.1617
Iteration 600, Loss: 0.1399
Iteration 700, Loss: 0.1768
Iteration 800, Loss: 0.1553
Iteration 900, Loss: 0.1338
Single Layer Perceptron Accuracy: 0.86


In [18]:
# EXP 5 Multi
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Create MLP model for regression
mlp = MLPRegressor(hidden_layer_sizes=(5,), max_iter=1000, random_state=42)

# Fit MLP model
mlp.fit(X_train, y_train)

# Predictions
mlp_predictions = mlp.predict(X_test)

# Calculate Mean Squared Error (MSE) for evaluation
mse = mean_squared_error(y_test, mlp_predictions)
print(f'Multi-Layer Perceptron MSE: {mse:.2f}')

# Optionally, you can calculate R^2 score to evaluate performance
r2_score = mlp.score(X_test, y_test)
print(f'Multi-Layer Perceptron R^2 Score: {r2_score:.2f}')


Multi-Layer Perceptron MSE: 0.09
Multi-Layer Perceptron R^2 Score: 0.55


In [10]:
# EXP 2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

# Initialize the Logistic Regression model
model = LogisticRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate Mean Squared Error for regression evaluation
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.1258794738452126


In [13]:
pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.5 MB/s eta 0:00:00


In [15]:
# EXP 3
from deap import base, creator, tools, algorithms
import random
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Create the fitness function (minimizing MSE)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Define the hyperparameter space
def create_individual():
    # Generate C as a positive float in the range (0.001, 100)
    C = random.uniform(0.001, 100)
    return [C]

# Fitness function to evaluate the individual
def evaluate(individual):
    C = individual[0]

    # Create and train the model with the current hyperparameter
    model = LogisticRegression(C=C, random_state=42)
    try:
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
        return (np.mean(-scores),)  # Return positive MSE
    except Exception as e:
        return (float('inf'),)  # Return a very high MSE if there's an error

# Initialize DEAP components
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Ensure that mutations do not generate out-of-bound values
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < 0.2:  # Mutation probability
            # Mutate C
            new_value = individual[i] + random.gauss(0, 0.1)  # Gaussian mutation
            # Clip to ensure the new value is within bounds
            individual[i] = max(0.001, min(100, new_value))
    return individual,

toolbox.register("mutate", mutate)

# Run the Genetic Algorithm
population = toolbox.population(n=10)
ngen = 10  # Number of generations
cxpb = 0.5  # Crossover probability
mutpb = 0.2  # Mutation probability

# Run the Genetic Algorithm
algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)

# Extract the best solution
best_ind = tools.selBest(population, k=1)[0]
print(f'Best Regularization Strength: {best_ind[0]}')



/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	10    
1  	8     
2  	6     
3  	6     
4  	8     
5  	7     
6  	2     
7  	8     
8  	5     
9  	7     
10 	6     
Best Regularization Strength: 3.0769056629418183


In [ ]:
# EXP 4
import random
import numpy as np
# Objective function
def objective_function(x):
 return x[0]**2 + x[1]**2 + 1
# PSO parameters
num_dimensions = 2
num_particles = 30
max_iterations = 100
# Initialize particles
particles = np.random.rand(num_particles, num_dimensions)
velocities = np.random.rand(num_particles, num_dimensions)
personal_best_positions = particles.copy()
personal_best_values = np.array([objective_function(x) for x in particles])
# Initialize global best
global_best_position = personal_best_positions[np.argmin(personal_best_values)]
global_best_value = min(personal_best_values)
# PSO loop
for iteration in range(max_iterations):
 for i in range(num_particles):
 # Update velocities
 velocities[i] = velocities[i] + \
 random.random() * (personal_best_positions[i] - particles[i]) + \
 random.random() * (global_best_position - particles[i])

 # Update particle positions
 particles[i] += velocities[i]

 # Update personal bests
 current_value = objective_function(particles[i])
 if current_value < personal_best_values[i]:
 personal_best_positions[i] = particles[i]
 personal_best_values[i] = current_value

 # Update global best
 if current_value < global_best_value:
 global_best_position = particles[i]
 global_best_value = current_value
print(f"Best position: {global_best_position}")
print(f"Best value: {global_best_value}")
